In [1]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 6)  # 5 class
model.load_state_dict(torch.load(r"D:\classified_music\efficientnet_b0_6_final_smoothing.pth", map_location=device))
model.to(device)
model.eval()

# Transform ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Class names
# class_names =  ['bolero', 'cailuong', 'cheo', 'danca', 'nhacdo']

d:\classified_music\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\classified_music\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
import os
import librosa as lb
import soundfile as sf
import numpy as np
import cv2
# class_names = ['CachMang', 'TruTinh', 'bolero', 'cailuong', 'cheo', 'danca', 'nhacxuan']
class_names =  ['bolero', 'cailuong', 'cheo', 'danca', 'other', 'thieunhi']
img_height,img_width=224,224

In [3]:
import time,random
import os
import json
import yt_dlp
from pydub import AudioSegment
import subprocess

def download_and_convert_youtube_ffmpeg(link, save_path_wav):
    temp_file = save_path_wav.replace('.wav', '.m4a')

    ydl_opts = {
        'format': 'bestaudio[ext=m4a]/bestaudio/best',
        'outtmpl': temp_file,
        'quiet': True,
        'cookiefile': 'cookie.txt'
    }

    try:
        # Download audio
        print(f"🎵 Đang tải: {link}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([link])
        
        if not os.path.exists(temp_file):
            raise FileNotFoundError(f"Không tìm thấy file: {temp_file}")

        # Convert bằng ffmpeg
        print(f"🎧 Đang chuyển đổi sang WAV...")
        cmd = [
            "ffmpeg", "-y",
            "-i", temp_file,
            "-ac", "1",               # mono
            "-ar", "16000",           # sample rate 16kHz
            save_path_wav
        ]
        subprocess.run(cmd, check=True)

        os.remove(temp_file)
        print(f"✅ Đã lưu WAV: {save_path_wav}")
    except Exception as e:
        print(f"❌ Lỗi với {link}: {e}")


In [4]:
import os
import librosa as lb
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import matplotlib.cm as cm
import torch.nn.functional as F
# Hàm xử lý STFT
def get_fft(samples, n_fft=2048, hop_length=512):
    for index, item in samples.items():
        D = np.abs(lb.stft(item["sampling"], n_fft=n_fft, hop_length=hop_length))
        samples[index]["stft"] = D
    return samples

# Hàm xử lý mel spectrogram
def get_mel_spectrogram(samples, sr=22050):
    for index, item in samples.items():
        S = lb.feature.melspectrogram(y=item["sampling"], sr=sr)
        S_db = lb.amplitude_to_db(S, ref=np.max)
        samples[index]["mel-spec-db"] = S_db
    return samples

# Hàm lưu ảnh mel spectrogram
# def save_mel_spec(samples, root):
#     image_paths = []
#     for index, item in samples.items():
#         S_db = item["mel-spec-db"]
#         os.makedirs(root, exist_ok=True)


#         file_name = os.path.splitext(os.path.basename(item["dir"]))[0]
#         out_path = os.path.join(root, file_name + ".png")
#         plt.imsave(out_path, S_db)
#         image_paths.append(out_path)
#     return image_paths
def save_mel_spec(samples, root):
    image_paths = []
    for index, item in samples.items():
        S_db = item["mel-spec-db"]
        os.makedirs(root, exist_ok=True)

        file_name = os.path.splitext(os.path.basename(item["dir"]))[0]
        out_path = os.path.join(root, file_name + ".png")

        # Normalize để đưa về 0–255
        S_db_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min())  # 0–1
        S_rgb = cm.viridis(S_db_norm)[:, :, :3]  # RGB, bỏ A
        S_rgb = (S_rgb * 255).astype(np.uint8)

        # Resize về 224x224 rồi lưu
        im = Image.fromarray(S_rgb).resize((224, 224))
        im.save(out_path)

        image_paths.append(out_path)

    return image_paths

# Hàm chính: predict và sinh ảnh spectrogram
def predict(file_path, duration=30):
    try:
        y, sr = lb.load(file_path, sr=None)
    except Exception as e:
        print(f" Lỗi khi load: {file_path} ({e})")
        return

    segment_samples = duration * sr
    total_samples = len(y)
    num_segments = total_samples // segment_samples
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    if num_segments == 0:
        return

    folder_path = os.path.dirname(file_path)

    # Folder lưu wav đã cắt
    output_folder = os.path.join(folder_path, "predict")
    os.makedirs(output_folder, exist_ok=True)

    samples = {}

    for i in range(num_segments):
        start = i * segment_samples
        end = start + segment_samples
        segment = y[start:end]

        new_filename = f"{base_filename}_part{i+1}.wav"
        new_path = os.path.join(output_folder, new_filename)

        sf.write(new_path, segment, sr)
        samples[i] = {
            "dir": new_path,
            "sampling": segment
        }

    samples = get_fft(samples)
    samples = get_mel_spectrogram(samples, sr)

    mel_root = os.path.join(output_folder, "mel-images")
    os.makedirs(mel_root, exist_ok=True)

    list_test = save_mel_spec(samples, mel_root)

    # === DỰ ĐOÁN TỪ ẢNH MEL ===
    print("\n🔍 Đang dự đoán")
    for path in list_test:
        image_pil = Image.open(path).convert("RGB") 
        image_pil = image_pil.resize((224, 224))
        # Apply transforms và chuyển sang tensor
        image_tensor = transform(image_pil).unsqueeze(0).to(device)

        # Dự đoán
        with torch.no_grad():
            outputs = model(image_tensor)
            probs = F.softmax(outputs, dim=1)  # chuyển logits thành xác suất
            conf, pred = torch.max(probs, 1)   # lấy class có xác suất cao nhất
        
            output_class = class_names[pred.item()]
            percentage = conf.item() * 100
            print(f"Ảnh {os.path.basename(path)} → Dự đoán: {output_class} ({percentage:.2f}%)")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/ncJF73mZ2SI?si=-ivhZC-_sFohG7Pi",r"D:\classified_music\data_test\cheo\test3.wav")

🎵 Đang tải: https://youtu.be/ncJF73mZ2SI?si=-ivhZC-_sFohG7Pi
🎧 Đang chuyển đổi sang WAV...                             
✅ Đã lưu WAV: D:\classified_music\data_test\cheo\test3.wav


In [5]:
predict(r"L:\Trung Tam SX\Audio Files\Giang_Ngoc_Productions\8935285213703\8935285213703_1_6.wav")

PermissionError: [WinError 5] Access is denied: 'L:\\Trung Tam SX\\Audio Files\\Giang_Ngoc_Productions\\8935285213703\\predict'

In [38]:
download_and_convert_youtube_ffmpeg("https://youtu.be/TEVjPTCoXAc?si=L4VKl4kMTyLK27XI",r"D:\classified_music\data_test\thieunhi\thieunhi_test6.wav")

🎵 Đang tải: https://youtu.be/TEVjPTCoXAc?si=L4VKl4kMTyLK27XI
🎧 Đang chuyển đổi sang WAV...                           
✅ Đã lưu WAV: D:\classified_music\data_test\thieunhi\thieunhi_test6.wav


In [6]:
predict(r"C:\Users\BHM-KyThuat\Desktop\Con heo đất.wav")


🔍 Đang dự đoán
Ảnh Con heo đất_part1.png → Dự đoán: thieunhi (88.05%)
Ảnh Con heo đất_part2.png → Dự đoán: thieunhi (88.27%)
Ảnh Con heo đất_part3.png → Dự đoán: thieunhi (90.85%)
Ảnh Con heo đất_part4.png → Dự đoán: thieunhi (81.28%)
Ảnh Con heo đất_part5.png → Dự đoán: thieunhi (80.44%)
Ảnh Con heo đất_part6.png → Dự đoán: thieunhi (88.87%)


In [ ]:
download_and_convert_youtube("https://youtu.be/pqZ7940oZVs?si=CFlDiV23IHgxBLkG",r"D:\classified_music\data_test\cheo\cheo_test1.wav")

In [8]:
predict(r"D:\classified_music\data_test\cheo\cheo_test1.wav")


🔍 Đang dự đoán
Ảnh cheo_test1_part1.png → Dự đoán: cheo (99.95%)
Ảnh cheo_test1_part2.png → Dự đoán: cheo (99.98%)
Ảnh cheo_test1_part3.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part4.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part5.png → Dự đoán: cheo (99.90%)
Ảnh cheo_test1_part6.png → Dự đoán: cheo (99.99%)
Ảnh cheo_test1_part7.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part8.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part9.png → Dự đoán: cheo (99.99%)
Ảnh cheo_test1_part10.png → Dự đoán: cheo (99.98%)
Ảnh cheo_test1_part11.png → Dự đoán: cheo (99.99%)
Ảnh cheo_test1_part12.png → Dự đoán: cheo (99.97%)
Ảnh cheo_test1_part13.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part14.png → Dự đoán: cheo (99.99%)
Ảnh cheo_test1_part15.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part16.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part17.png → Dự đoán: cheo (100.00%)
Ảnh cheo_test1_part18.png → Dự đoán: cheo (99.78%)
Ảnh cheo_test1_part19.png → Dự đoán: cheo (98.42%)
Ảnh cheo_test1_p

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/L2nvb9QveRM?si=qfaZtvVujwGzI1CT",r"D:\classified_music\data_test\cailuong\cailuong_test1.wav")

In [ ]:
predict("D:\classified_music\data_test\dan ca\Em_Về_Miệt_Thứ_Hương_Lan.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/dLNZRV0JhA4?si=x-28L0IjWTnYs50W",r"D:\classified_music\data_test\cailuong\cailuong_xua.wav")

In [ ]:
predict("D:\classified_music\data_test\cailuong\cailuong_xua.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/SmIA6O7G01s?si=KH6tR8i-fN9BxnpH",r"D:\classified_music\data_test\bolero\bolero_test1.wav")

In [ ]:
predict(r"D:\classified_music\data_test\bolero\bolero_test1.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/_bh1t8zmW80?si=jHGnnKiGuXXmUII8",r"D:\classified_music\data_test\bolero\bolero_test2.wav")

In [ ]:
predict(r"D:\classified_music\data_test\bolero\bolero_test2.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/elV7q58D6w4?si=2e5LseESxrgzad53",r"D:\classified_music\data_test\nhac xuan\xuan_test1.wav")

In [ ]:
predict(r"D:\classified_music\data_test\nhac xuan\xuan_test1.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/HIu5-z0_lzs?si=PcLd4nSlEAfPFr_L",r"D:\classified_music\data_test\nhac_do\test1.wav")

In [ ]:
predict(r"D:\classified_music\data_test\nhac_do\test1.wav")

In [ ]:
download_and_convert_youtube_ffmpeg("https://youtu.be/2EBWjBMTfUo?si=YE7lHgnwbuwtHEj1",r"D:\classified_music\data_test\pop\test.wav")

In [9]:
predict(r"D:\classified_music\data_test\pop\test.wav")


🔍 Đang dự đoán
Ảnh test_part1.png → Dự đoán: nhacdo (92.02%)
Ảnh test_part2.png → Dự đoán: nhacdo (98.25%)
Ảnh test_part3.png → Dự đoán: nhacdo (94.89%)
Ảnh test_part4.png → Dự đoán: nhacdo (78.44%)
Ảnh test_part5.png → Dự đoán: nhacdo (97.97%)


In [9]:
download_and_convert_youtube_ffmpeg("https://youtu.be/EyvBWee-ECw?si=1x7K_u7tDRewnVRp",r"D:\classified_music\data_test\thieunhi\thieunhi_test5.wav")

🎵 Đang tải: https://youtu.be/EyvBWee-ECw?si=1x7K_u7tDRewnVRp
🎧 Đang chuyển đổi sang WAV...                             
✅ Đã lưu WAV: D:\classified_music\data_test\thieunhi\thieunhi_test5.wav


In [10]:
download_and_convert_youtube_ffmpeg("https://youtu.be/hzgmksoMExQ?si=CIp2RWzlZGgRDE7x",r"D:\classified_music\data_test\remix.wav")

🎵 Đang tải: https://youtu.be/hzgmksoMExQ?si=CIp2RWzlZGgRDE7x
🎧 Đang chuyển đổi sang WAV...                             
✅ Đã lưu WAV: D:\classified_music\data_test\remix.wav


In [12]:
predict(r"D:\classified_music\data_test\remix.wav")


🔍 Đang dự đoán
Ảnh remix_part1.png → Dự đoán: nhacdo (98.63%)
Ảnh remix_part2.png → Dự đoán: nhacdo (97.90%)
Ảnh remix_part3.png → Dự đoán: nhacdo (99.99%)
Ảnh remix_part4.png → Dự đoán: nhacdo (99.62%)
Ảnh remix_part5.png → Dự đoán: nhacdo (99.95%)
Ảnh remix_part6.png → Dự đoán: nhacdo (93.71%)
Ảnh remix_part7.png → Dự đoán: nhacdo (99.92%)
Ảnh remix_part8.png → Dự đoán: nhacdo (100.00%)
Ảnh remix_part9.png → Dự đoán: nhacdo (99.99%)
Ảnh remix_part10.png → Dự đoán: nhacdo (100.00%)
Ảnh remix_part11.png → Dự đoán: nhacdo (100.00%)
Ảnh remix_part12.png → Dự đoán: nhacdo (100.00%)
Ảnh remix_part13.png → Dự đoán: cailuong (78.99%)
Ảnh remix_part14.png → Dự đoán: nhacdo (74.60%)
Ảnh remix_part15.png → Dự đoán: nhacdo (99.94%)
Ảnh remix_part16.png → Dự đoán: nhacdo (99.27%)
Ảnh remix_part17.png → Dự đoán: nhacdo (84.96%)
Ảnh remix_part18.png → Dự đoán: nhacdo (99.90%)
Ảnh remix_part19.png → Dự đoán: nhacdo (99.27%)
Ảnh remix_part20.png → Dự đoán: nhacdo (96.23%)
Ảnh remix_part21.png → Dự đ